In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

#### 1a. Data pre processing - Example

##### 1.Read the data and exploration

In [16]:
# Read the data
cust_data = pd.read_csv('CustomerLoanData.csv')

In [17]:
# Sample data
cust_data.head()

,age,exp,inc,family,edu,mortgage,ccAvg,securities,cd,online,cc,infoReq,loan
0,NaN,1.0,49.0,4.0,1.0,0.0,20.075206,1.0,0.0,0.0,0.0,Email,0
1,45.0,19.0,34.0,3.0,1.0,0.0,18.879367,1.0,0.0,0.0,0.0,Email,0
2,39.0,15.0,11.0,1.0,1.0,0.0,13.074596,0.0,0.0,0.0,0.0,Email,0
3,35.0,9.0,100.0,1.0,2.0,0.0,35.069547,0.0,0.0,0.0,0.0,Email,0
4,35.0,8.0,45.0,4.0,2.0,0.0,13.044593,0.0,0.0,0.0,1.0,Email,0


In [18]:
# Dimensions of data
cust_data.shape

(5000, 13)

In [19]:
# Column names of data
cust_data.columns

Index(['age', 'exp', 'inc', 'family', 'edu', 'mortgage', 'ccAvg', 'securities',
       'cd', 'online', 'cc', 'infoReq', 'loan'],
      dtype='object')

In [20]:
# Structure of data
cust_data.dtypes

age           float64
exp           float64
inc           float64
family        float64
edu           float64
mortgage      float64
ccAvg         float64
securities    float64
cd            float64
online        float64
cc            float64
infoReq        object
loan            int64
dtype: object

In [21]:
# Summary of data
cust_data.describe()

,age,exp,inc,family,edu,mortgage,ccAvg,securities,cd,online,cc,loan
count,4727.000000,4690.000000,4705.000000,4717.000000,4728.000000,4706.00000,5000.000000,4849.000000,4853.000000,4878.000000,4864.000000,5000.000000
mean,45.350751,20.169936,73.726249,2.399406,1.876481,56.60051,25.030919,0.102702,0.059551,0.595941,0.292558,0.096000
std,11.485832,11.447317,45.961482,1.148026,0.838802,102.04044,22.239978,0.303600,0.236677,0.490759,0.454984,0.294621
min,23.000000,-3.000000,8.000000,1.000000,1.000000,0.00000,0.984891,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.000000,10.000000,39.000000,1.000000,1.000000,0.00000,8.938454,0.000000,0.000000,0.000000,0.000000,0.000000
50%,45.000000,20.000000,64.000000,2.000000,2.000000,0.00000,19.238094,0.000000,0.000000,1.000000,0.000000,0.000000
75%,55.000000,30.000000,98.000000,3.000000,3.000000,101.00000,32.489010,0.000000,0.000000,1.000000,1.000000,0.000000
max,67.000000,43.000000,224.000000,4.000000,3.000000,635.00000,108.398884,1.000000,1.000000,1.000000,1.000000,1.000000


##### 2.Train & test split

In [22]:
trainx,testx,trainy,testy = train_test_split(cust_data.iloc[:,:-1],cust_data.iloc[:,-1],test_size=0.3,random_state=1)
print(cust_data.shape)
print(trainx.shape)
print(testx.shape)

(5000, 13)
(3500, 12)
(1500, 12)


##### 3. Data type conversion

In [23]:
cat_cols = ["edu","securities","cd","online","cc","infoReq"]
num_cols = trainx.columns.difference(cat_cols)
num_cols

Index(['age', 'ccAvg', 'exp', 'family', 'inc', 'mortgage'], dtype='object')

In [24]:
trainx[cat_cols] = trainx[cat_cols].apply(lambda x: x.astype('category'))
trainx[num_cols] = trainx[num_cols].apply(lambda x: x.astype('float'))
trainx.dtypes

age            float64
exp            float64
inc            float64
family         float64
edu           category
mortgage       float64
ccAvg          float64
securities    category
cd            category
online        category
cc            category
infoReq       category
dtype: object

##### 4. Imputation

In [25]:
trainx.isnull().sum()

age           196
exp           202
inc           193
family        193
edu           187
mortgage      207
ccAvg           0
securities    108
cd             98
online         83
cc             90
infoReq         0
dtype: int64

In [26]:
num_data = trainx.loc[:,num_cols]
cat_data = trainx.loc[:,cat_cols]

In [27]:
# Numeric columns imputation
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
num_data = pd.DataFrame(imp.fit_transform(num_data),columns=num_cols)

# Categorical columns imputation
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
cat_data = pd.DataFrame(imp.fit_transform(cat_data),columns=cat_cols)


print(num_data.isnull().sum())
print(cat_data.isnull().sum())

age         0
ccAvg       0
exp         0
family      0
inc         0
mortgage    0
dtype: int64
edu           0
securities    0
cd            0
online        0
cc            0
infoReq       0
dtype: int64


##### 5. Standardization

In [28]:
standardizer = StandardScaler()
standardizer.fit(num_data)
num_data = pd.DataFrame(standardizer.transform(num_data),columns=num_cols)

trainx = pd.concat([num_data,cat_data],axis=1)

##### 6. Binning

In [29]:
bins = [trainx.ccAvg.min(),np.median(trainx.ccAvg),trainx.ccAvg.max()]
group_names = ['low', 'high']
trainx['cat_cc'] = pd.cut(trainx['ccAvg'],bins, labels=group_names)
trainx.head()

,age,ccAvg,exp,family,inc,mortgage,edu,securities,cd,online,cc,infoReq,cat_cc
0,0.136272,-0.408423,0.153558,-0.356562,-0.862109,-0.574717,1,0,0,1,0,Email,low
1,-0.667571,0.044312,-0.562882,-1.254621,-0.772713,-0.574717,2,0,0,1,0,Email,high
2,1.208062,1.118236,1.317771,-1.254621,1.283395,-0.574717,1,0,0,1,1,Email,high
3,-1.471414,-0.054355,-1.547986,1.439555,-0.191639,1.927875,2,0,0,0,0,Email,high
4,-0.578255,-0.997399,-0.652437,0.541496,0.000000,-0.574717,2,0,0,1,0,Ph,low


##### 7. Creating dummies

In [30]:
trainx=pd.get_dummies(trainx,columns=cat_cols.extend(['cat_cc']))

In [31]:
trainx.head()

,age,ccAvg,exp,family,inc,mortgage,edu_1.0,edu_2.0,edu_3.0,securities_0.0,...,cd_0.0,cd_1.0,online_0.0,online_1.0,cc_0.0,cc_1.0,infoReq_Email,infoReq_Ph,cat_cc_low,cat_cc_high
0,0.136272,-0.408423,0.153558,-0.356562,-0.862109,-0.574717,1,0,0,1,...,1,0,0,1,1,0,1,0,1,0
1,-0.667571,0.044312,-0.562882,-1.254621,-0.772713,-0.574717,0,1,0,1,...,1,0,0,1,1,0,1,0,0,1
2,1.208062,1.118236,1.317771,-1.254621,1.283395,-0.574717,1,0,0,1,...,1,0,0,1,0,1,1,0,0,1
3,-1.471414,-0.054355,-1.547986,1.439555,-0.191639,1.927875,0,1,0,1,...,1,0,1,0,1,0,1,0,0,1
4,-0.578255,-0.997399,-0.652437,0.541496,0.000000,-0.574717,0,1,0,1,...,1,0,0,1,1,0,0,1,1,0


#### 1a. Data pre processing - Activity

1. Read the data 'train.csv'
    1. Split the data into train and test
2. Do the folowing pre processing steps on train:
    1. Find the un wanted columns and remove them from train
    2. Identify the column types and convert them appropriately
    3. Impute the data
    4. Standardize the data
    5. Get dummies
3. Do the following preprocessing on test data:
    1. Keeping only the required columns
    2. Convert the data types
    3. Impute the data
    4. Standardize the data
    5. Get dummies

##### 1. Read the data and divide into train and test

In [32]:
cab_data = pd.read_csv('train.csv')
trainx,testx,trainy,testy = train_test_split(cab_data.iloc[:,:-1],cab_data.iloc[:,-1],test_size=0.3,random_state=1)
print(cab_data.shape)
print(trainx.shape)
print(testx.shape)

(119386, 18)
(83570, 17)
(35816, 17)


In [33]:
cab_data.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,surcharge,fare_amount
0,AIX000345005,DST000401,NO,0.0,0.0,0.5,2015-04-15 13:12:48,2015-04-15 13:26:30,1,-74.004657,40.707434,1,N,-73.999369,40.721517,CSH,0.0,10.0
1,AIX000345019,DST000401,NO,0.0,0.0,0.5,2015-04-18 20:29:50,2015-04-18 20:34:15,1,-73.995108,40.755005,1,N,-73.989492,40.745640,CSH,0.5,6.5
2,AIX000345026,DST000401,NO,0.0,NaN,0.5,2015-04-07 09:33:16,2015-04-07 09:42:16,1,-73.969530,40.790267,1,N,-73.987282,40.766865,CRD,0.0,10.6
3,AIX000345034,DST000401,NO,0.0,1.7,0.5,2015-04-23 23:22:45,2015-04-23 23:28:19,1,-73.977336,40.742819,1,N,-73.993617,40.721326,CRD,0.5,10.2
4,AIX000345052,DST000401,NO,0.0,1.0,0.5,2015-04-17 01:48:03,2015-04-17 01:53:32,1,-73.998653,40.715607,1,N,-73.988385,40.723130,CRD,0.5,8.5


###### 2. Preprocessing on train

Dropping the un wanted columns

In [34]:
cols_drop = ['TID','pickup_datetime','dropoff_datetime',
              'pickup_longitude','pickup_latitude',
              'dropoff_longitude', 'dropoff_latitude']

In [35]:
trainx.drop(cols_drop,axis=1,inplace=True)

In [36]:
trainx.head()

,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,passenger_count,rate_code,store_and_fwd_flag,payment_type,surcharge
96235,DST000543,NO,0.0,2.49,0.5,5,1,N,CRD,0.0
52381,DST000532,NO,0.0,1.90,0.5,5,1,NaN,CRD,0.5
42657,DST000532,NO,0.0,0.00,0.5,1,1,NaN,CSH,0.0
109228,DST000481,NO,0.0,1.55,0.5,1,1,N,CRD,1.0
79660,DST000532,NO,0.0,2.40,0.5,1,1,NaN,UNK,0.0


Convert the data types appropriately

In [37]:
cat_cols = ["vendor_id","new_user","store_and_fwd_flag","payment_type"]
num_cols = trainx.columns.difference(cat_cols)
num_cols

Index(['mta_tax', 'passenger_count', 'rate_code', 'surcharge', 'tip_amount',
       'tolls_amount'],
      dtype='object')

In [38]:
trainx[cat_cols] = trainx[cat_cols].apply(lambda x: x.astype('category'))
trainx[num_cols] = trainx[num_cols].apply(lambda x: x.astype('float'))
trainx.dtypes

vendor_id             category
new_user              category
tolls_amount           float64
tip_amount             float64
mta_tax                float64
passenger_count        float64
rate_code              float64
store_and_fwd_flag    category
payment_type          category
surcharge              float64
dtype: object

Imputation

In [39]:
train_num_data = trainx.loc[:,num_cols]
train_cat_data = trainx.loc[:,cat_cols]

In [40]:
# Numeric columns imputation
imp_num = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_num.fit(train_num_data)
train_num_data = pd.DataFrame(imp_num.transform(train_num_data),columns=num_cols)

# Categorical columns imputation
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_cat_data = pd.DataFrame(imp.fit_transform(train_cat_data),columns=cat_cols)

Standardization

In [41]:
standardizer = StandardScaler()
standardizer.fit(train_num_data)
train_num_data = pd.DataFrame(standardizer.transform(train_num_data),columns=num_cols)

trainx = pd.concat([train_num_data,train_cat_data],axis=1)

Creating dummies

In [42]:
trainx=pd.get_dummies(trainx,columns=cat_cols)

In [43]:
trainx.head()

,mta_tax,passenger_count,rate_code,surcharge,tip_amount,tolls_amount,vendor_id_DST000401,vendor_id_DST000481,vendor_id_DST000532,vendor_id_DST000543,new_user_NO,new_user_YES,store_and_fwd_flag_N,store_and_fwd_flag_Y,payment_type_CRD,payment_type_CSH,payment_type_DIS,payment_type_NOC,payment_type_UNK
0,0.060661,2.366174,-0.127424,-0.902658,0.426615,-0.212917,0,0,0,1,1,0,1,0,1,0,0,0,0
1,0.060661,2.366174,-0.127424,0.523219,0.159844,-0.212917,0,0,1,0,1,0,1,0,1,0,0,0,0
2,0.060661,-0.519421,-0.127424,-0.902658,-0.699246,-0.212917,0,0,1,0,1,0,1,0,0,1,0,0,0
3,0.060661,-0.519421,-0.127424,1.949095,0.001591,-0.212917,0,1,0,0,1,0,1,0,1,0,0,0,0
4,0.060661,-0.519421,-0.127424,-0.902658,0.385921,-0.212917,0,0,1,0,1,0,1,0,0,0,0,0,1


##### 3. Preprocessing on test

In [44]:
testx.drop(cols_drop,axis=1,inplace=True)
testx[cat_cols] = testx[cat_cols].apply(lambda x: x.astype('category'))
testx[num_cols] = testx[num_cols].apply(lambda x: x.astype('float'))


test_num_data = testx.loc[:,num_cols]
test_cat_data = testx.loc[:,cat_cols]

test_num_data = pd.DataFrame(imp_num.transform(test_num_data),columns=num_cols)
test_cat_data = pd.DataFrame(imp.transform(test_cat_data),columns=cat_cols)

test_num_data = pd.DataFrame(standardizer.transform(test_num_data),columns=num_cols)

testx = pd.concat([test_num_data,test_cat_data],axis=1)

testx = pd.get_dummies(testx,columns=cat_cols)

testx.head()

,mta_tax,passenger_count,rate_code,surcharge,tip_amount,tolls_amount,vendor_id_DST000401,vendor_id_DST000481,vendor_id_DST000532,vendor_id_DST000543,new_user_NO,new_user_YES,store_and_fwd_flag_N,store_and_fwd_flag_Y,payment_type_CRD,payment_type_CSH,payment_type_DIS,payment_type_NOC,payment_type_UNK
0,0.060661,0.201978,-0.127424,0.523219,0.521567,-0.212917,0,0,1,0,1,0,1,0,1,0,0,0,0
1,0.060661,-0.519421,-0.127424,-0.902658,-0.699246,-0.212917,0,0,1,0,1,0,1,0,0,1,0,0,0
2,0.060661,-0.519421,-0.127424,1.949095,-0.699246,-0.212917,1,0,0,0,1,0,1,0,0,1,0,0,0
3,0.060661,-0.519421,-0.127424,0.000000,3.618815,-0.212917,0,1,0,0,1,0,1,0,1,0,0,0,0
4,0.060661,-0.519421,-0.127424,1.949095,-0.699246,-0.212917,0,0,0,1,1,0,1,0,0,1,0,0,0
